In [127]:
import pandas as pd
import numpy as np

In [128]:
df_ds = pd.read_excel('final_predictions/deepseek14_all_pred.xlsx', index_col=0)
df_llama = pd.read_excel('final_predictions/llama8_all_pred.xlsx', index_col=0)
df_gemma = pd.read_excel('final_predictions/gemma12_all_pred.xlsx', index_col=0)

In [129]:
col_labels = [col for col in df_ds.columns if col.startswith('label')]

## Determine the number of correct and incorrect predictions given the batch size, task, model, and dataset

In [ ]:

def get_stats_pred(df_ds, df_llama, df_gemma, task):
    df_all = {}
    df_3 = []
    selected_data = ['l', 'p', 'o', 'rds']
    model_names = []
    for i, df in enumerate([df_ds.reset_index(), df_llama.reset_index(), df_gemma.reset_index()]):
        model_names.append(df.iloc[0]['model'][:4])
            
        if task == 'f': real_col = 'IsFunctional'
        else: real_col = 'IsQuality'
        df_task = df[(df['Task'] == task) & (df['Dataset'].isin(selected_data))].copy()

        df_task.loc[:,'both'] = [1 if row['IsFunctional'] + row['IsQuality'] == 2 else 0 for i, row in df_task.iterrows()]
        

        df_task['correct_1'] = [1 if row['label_1'] == row[real_col] else 0 for i, row in df_task.iterrows()]
        df_task['correct_2'] = [1 if row['label_2'] == row[real_col] else 0 for i, row in df_task.iterrows()]
        df_task['correct_4'] = [1 if row['label_4'] == row[real_col] else 0 for i, row in df_task.iterrows()]
        df_task['correct_8'] = [1 if row['label_8'] == row[real_col] else 0 for i, row in df_task.iterrows()]
        df_task['correct_16'] = [1 if row['label_16'] == row[real_col] else 0 for i, row in df_task.iterrows()]
        df_task['correct_32'] = [1 if row['label_32'] == row[real_col] else 0 for i, row in df_task.iterrows()]
        df_task['correct_64'] = [1 if row['label_64'] == row[real_col] else 0 for i, row in df_task.iterrows()]
        if len(df_3) == 0:
            df_3 = df_task[['index',real_col,'RequirementText',  'both',
                            'Dataset', 'correct_1', 'correct_2','correct_4','correct_8','correct_16','correct_32','correct_64']]
                
        else:
            df_3 = pd.merge(df_3, df_task[['index','RequirementText', 'correct_1', 'correct_2','correct_4','correct_8','correct_16','correct_32','correct_64']], on = ['index', 'RequirementText'], suffixes=('_'+ model_names[i-1], '_'+ model_names[i]))
    return df_3

def agg_stats(df):
    batch_sizes = [1,2,4,8,16,32,64]
    all_stats = []
    for m in ['_deep', '_meta', '']:
        all_stats_per_model = []
        for bs in batch_sizes:
            df_i = df.groupby(['correct_' + str(bs)+ m, 'Dataset']).agg({'both': 'sum', 'RequirementText':'count'}).reset_index()
            df_i['type'] = bs

            if m == '':
                df_i['model'] = 'gemma'
            else:
                df_i['model'] = m[1:]
            
            df_i = df_i.rename(columns={'correct_' + str(bs)+ m: 'correct'})
            all_stats_per_model.append(df_i)
           
        df_stats_per_model = pd.concat(all_stats_per_model) 
       
        df_stats_per_model['ratio'] = df_stats_per_model['both']/df_stats_per_model['RequirementText']

        all_stats.append(df_stats_per_model)

    return pd.concat(all_stats)
         

In [131]:
df_q = get_stats_pred(df_ds, df_llama, df_gemma, 'q')
df_stats_quality = agg_stats(df_q)
df_f = get_stats_pred(df_ds, df_llama, df_gemma, 'f')
df_stats_functional = agg_stats(df_f)

## Calculate the ratio

In [ ]:
ratio_per_dataset = df_f.groupby('Dataset')['both'].aggregate(['sum', 'count']).reset_index()

In [ ]:
ratio_per_dataset['ratio_total'] = ratio_per_dataset['sum']/ratio_per_dataset['count']

In [111]:
df_stats_functional = df_stats_functional.merge(ratio_per_dataset[['Dataset', 'ratio_total']], on = 'Dataset')
df_stats_quality = df_stats_quality.merge(ratio_per_dataset[['Dataset', 'ratio_total']], on = 'Dataset')

## Ratio requirements with both characters

In [ ]:
df_stats_quality['norm_ratio2'] = df_stats_quality['ratio']/df_stats_quality['ratio_total']
df_stats_functional['norm_ratio2'] = df_stats_functional['ratio']/df_stats_functional['ratio_total']

In [115]:
df_stats_quality_0 = df_stats_quality[df_stats_quality['correct'] == 0] 
df_stats_functional_0 = df_stats_functional[df_stats_functional['correct'] == 0]

In [116]:
pivot_quality_0 =round(df_stats_quality_0.pivot_table(index=['type'], values=['norm_ratio2'],columns=['model','Dataset']),2)
pivot_quality_0.columns = pivot_quality_0.columns.swaplevel(0, 1) 
pivot_quality_0 = pivot_quality_0.sort_index(axis=1)
pivot_functional_0 =round(df_stats_functional_0.pivot_table(index=['type'], values=['norm_ratio2'],columns=['model','Dataset']),2)
pivot_functional_0.columns = pivot_functional_0.columns.swaplevel(0, 1) 
pivot_functional_0 = pivot_functional_0.sort_index(axis=1)

 ## Accuracy per dataset

In [119]:
df_stats_quality_1 = df_stats_quality[df_stats_quality['correct'] == 1] 
df_stats_functional_1 = df_stats_functional[df_stats_functional['correct'] == 1] 

In [ ]:
df_stats_quality_1.loc[:, ['rows']] = [64 if d == 'l' else 128 if d == 'o' else 192 if d in ['p','rds'] else -1 for d in df_stats_quality_1['Dataset']]
df_stats_functional_1.loc[:, ['rows']] = [64 if d == 'l' else 128 if d == 'o' else 192 if d in ['p','rds'] else -1 for d in df_stats_functional_1['Dataset']]

In [124]:
df_stats_quality_1.loc[:, ['acc']] = df_stats_quality_1['RequirementText']/df_stats_quality_1['rows'] 
df_stats_datasets_q = df_stats_quality_1.groupby(['model', 'Dataset'])['acc'].agg(['mean', 'std', 'min', 'max']).reset_index()

df_stats_functional_1.loc[:, ['acc']] = df_stats_functional_1['RequirementText']/df_stats_functional_1['rows']
df_stats_datasets_f = df_stats_functional_1.groupby(['model', 'Dataset'])['acc'].agg(['mean', 'std', 'min', 'max']).reset_index()

In [ ]:
with pd.ExcelWriter('pivottable_stats.xlsx', engine='xlsxwriter') as writer:
    pivot_quality_0.to_excel(writer, sheet_name='Quality')
    pivot_functional_0.to_excel(writer, sheet_name='Functional')
    round(df_stats_datasets_q.pivot_table(index='model', columns='Dataset', values=['mean','std', 'min', 'max']), 2).to_excel(writer, sheet_name='Acc_quality')
    round(df_stats_datasets_f.pivot_table(index='model', columns='Dataset', values=['mean','std', 'min', 'max']), 2).to_excel(writer, sheet_name='Acc_functional')